In [ ]:
# Libraries to import
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

# Creates a custom palette of Lifted colours based on RBG codes. To be used in matplotlib plots
lifted_palette = ['#7FDCED', '#4D00C0', '#00686B']

# Aggregating functions passed into dictionary to pass into aggregate method. Data being grouped by date and then
# aggregated. This removes all the Nan as in the initial download each measurement has its own row with 4 Nans
aggregating_functions = {'Sleep': 'sum', 'Pain': 'sum', 'Bowel Movements': 'sum', 'Meals Eaten': 'sum', 'Mood': 'sum'}

# Initialising an empty list to append client names to. This list will then be interated through to create a DF
# and graph for each client. Each DF for each client should then be added to the dictionary
client_names = []
client_dict = {}

# CHOOSE CSV FILE
df = pd.read_csv()

# To look up different measurements taken and then put those categories into new columns
# In the download from the admin app all categories and values are listed in one column
df['type'].value_counts()

df['Sleep'] = df[df['type']=='Sleep']['rating']
df['Pain'] = df[df['type']=='Pain']['rating']
df['Bowel Movements'] = df[df['type']=='Bowel Movements']['rating']
df['Meals Eaten'] = df[df['type']=='Meals Eaten']['rating']
df['Mood'] = df[df['type']=='Mood']['rating']

# Creates a new Date Time format column off of the visit 'Start' date and time
df['Date Time'] = pd.to_datetime(df['start'])

# Remove columns that aren't needed
df.drop('Carer', axis=1, inplace=True)
df.drop('start', axis=1, inplace=True)
df.drop('type', axis=1, inplace=True)
df.drop('rating', axis=1, inplace=True)

In [ ]:
client_names = df['Client'].unique()
client_names

# Passes is a numpy array of client names to iterate through. Creates a new DF and adds to dictionary of DFs. Key is client
# name
def client_df_creation(clients):
    for client in clients:
        client_df = df[df['Client']== client]
        # Sets datetime column as the DF index
        client_df = client_df.set_index('Date Time')
        # Resample index from daily to weekly data and gives the mean value for each measurement
        client_df = client_df.resample('W').mean()
        # Adds new DF to dictionary of DFs
        client_dict[client] = client_df 
    return client_dict

In [ ]:
# Gets list of keys from client_dict to pass into function to then plot off of
client_dict_keys = list(client_dict.keys())

def plotting_func(keys):
    for key in keys:
        summary_fig = plt.figure(figsize=(16,6), dpi=200)

        # Add set of axes to figure
        ax = summary_fig.add_axes([0.1, 0.1, 0.8, 0.8]) # left, bottom, width, height (range 0 to 1)

        # Set axis and title lables
        ax.set_xlabel('Time', fontsize='20', color='#708090', alpha=0.8)
        ax.set_ylabel('Rating', fontsize='20', color='#708090', alpha=0.8)
        ax.set_title('{} Wellness Scores'.format(key), fontsize='30', color='#708090', alpha=0.8)

        ax.plot(client_dict[key].index, client_dict[key]['Sleep']*2, marker='o', fillstyle='left', 
                label='Sleep', color='cornflowerblue')
        ax.plot(client_dict[key].index, client_dict[key]['Pain']*2, marker='o', fillstyle='left',
               label='Pain', color='darkorange')
        ax.plot(client_dict[key].index, client_dict[key]['Bowel Movements']*2, marker='o', 
                fillstyle='left', label='Bowel Movements', color='darkseagreen')
        ax.plot(client_dict[key].index, client_dict[key]['Meals Eaten']*2, marker='o', fillstyle='left',
               label='Meals Eaten', color='tomato')
        ax.plot(client_dict[key].index, client_dict[key]['Mood']*2, marker='o', fillstyle='left',
               label='Mood', color='orchid')

        # Background Grid for Plot
        ax.grid(color='#4D00C0', alpha=0.2)

        # To hide right and top border
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)

        ax.legend()
        
        summary_fig.savefig('{}, {}.pdf'.format(key, datetime.now().strftime("%Y-%m-%d")))
        